# Pagbuo gamit ang Mistral Models

## Panimula

Tatalakayin sa araling ito ang:
- Pag-explore sa iba't ibang Mistral Models
- Pag-unawa sa mga gamit at sitwasyon para sa bawat modelo
- Mga halimbawa ng code na nagpapakita ng natatanging katangian ng bawat modelo


## Ang Mga Mistral Model

Sa araling ito, tatalakayin natin ang 3 iba’t ibang Mistral model: **Mistral Large**, **Mistral Small** at **Mistral Nemo**.

Lahat ng mga modelong ito ay libre at makukuha sa Github Model marketplace. Ang code sa notebook na ito ay gagamit ng mga modelong ito para patakbuhin ang code. Narito ang karagdagang detalye tungkol sa paggamit ng Github Models para [mag-prototype gamit ang AI models](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Ang Mistral Large 2 ang kasalukuyang pangunahing modelo mula sa Mistral at idinisenyo para sa paggamit ng mga negosyo.

Ang modelong ito ay isang pag-upgrade mula sa orihinal na Mistral Large sa pamamagitan ng pag-aalok ng
-  Mas Malaking Context Window - 128k kumpara sa 32k
-  Mas mahusay na performance sa Math at Coding Tasks - 76.9% average accuracy kumpara sa 60.4%
-  Pinahusay na multilingual na performance - kabilang ang mga wika tulad ng: Ingles, Pranses, Aleman, Espanyol, Italyano, Portuges, Olandes, Ruso, Tsino, Hapones, Koreano, Arabe, at Hindi.

Dahil sa mga tampok na ito, namumukod-tangi ang Mistral Large sa
- *Retrieval Augmented Generation (RAG)* - dahil sa mas malaking context window
- *Function Calling* - ang modelong ito ay may native na function calling na nagbibigay-daan sa integrasyon sa mga external na tool at API. Maaaring gawin ang mga tawag na ito nang sabay-sabay o sunud-sunod.
- *Code Generation* - mahusay ang modelong ito sa pagbuo ng Python, Java, TypeScript, at C++.


Sa halimbawang ito, ginagamit natin ang Mistral Large 2 para magpatakbo ng RAG pattern sa isang text na dokumento. Ang tanong ay nakasulat sa Korean at nagtatanong tungkol sa mga aktibidad ng may-akda bago pumasok sa kolehiyo.

Gumagamit ito ng Cohere Embeddings Model para gumawa ng embeddings ng text na dokumento pati na rin ng tanong. Para sa halimbawang ito, ginagamit nito ang faiss Python package bilang vector store.

Kasama sa prompt na ipinapadala sa Mistral model ang parehong mga tanong at ang mga nakuha na bahagi ng teksto na kahawig ng tanong. Pagkatapos, nagbibigay ang Model ng sagot gamit ang natural na wika.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small
Ang Mistral Small ay isa pang modelo sa pamilya ng Mistral na kabilang sa premier/enterprise na kategorya. Gaya ng ipinapahiwatig ng pangalan, ito ay isang Small Language Model (SLM). Ang mga benepisyo ng paggamit ng Mistral Small ay:
- Mas matipid kumpara sa mga Mistral LLM tulad ng Mistral Large at NeMo – 80% na pagbaba ng presyo
- Mababa ang latency – mas mabilis ang tugon kumpara sa mga LLM ng Mistral
- Flexible – maaaring i-deploy sa iba’t ibang environment na may mas kaunting limitasyon sa kinakailangang resources.

Ang Mistral Small ay mahusay para sa:
- Mga gawain na nakabase sa teksto tulad ng pagbuod, pagsusuri ng damdamin, at pagsasalin.
- Mga aplikasyon na madalas ang request dahil sa pagiging cost effective nito
- Mga code task na nangangailangan ng mababang latency tulad ng review at code suggestions


## Paghahambing ng Mistral Small at Mistral Large

Para ipakita ang pagkakaiba ng latency sa pagitan ng Mistral Small at Large, patakbuhin ang mga cell sa ibaba.

Makikita mo ang pagkakaiba sa bilis ng sagot na nasa 3-5 segundo. Pansinin din ang haba at istilo ng sagot gamit ang parehong prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Kung ikukumpara sa dalawang modelong tinalakay sa araling ito, ang Mistral NeMo lang ang libreng modelo na may Apache2 License.

Itinuturing itong upgrade mula sa naunang open source LLM ng Mistral, ang Mistral 7B.

Ilan pa sa mga tampok ng NeMo model ay:

- *Mas episyenteng tokenization:* Ginagamit ng modelong ito ang Tekken tokenizer imbes na ang mas karaniwang tiktoken. Dahil dito, mas maganda ang performance nito sa mas maraming wika at code.

- *Finetuning:* Ang base model ay available para sa finetuning. Nagbibigay ito ng mas malaking flexibility para sa mga sitwasyon na nangangailangan ng finetuning.

- *Native Function Calling* - Katulad ng Mistral Large, sinanay din ang modelong ito sa function calling. Natatangi ito dahil isa ito sa mga unang open source models na may ganitong kakayahan.


## Mistral NeMo

Kung ikukumpara sa dalawang modelong tinalakay sa araling ito, ang Mistral NeMo lang ang libreng modelo na may Apache2 License.

Itinuturing itong mas pinahusay na bersyon ng naunang open source LLM mula sa Mistral, ang Mistral 7B.

Ilan pa sa mga tampok ng NeMo model ay:

- *Mas episyenteng tokenization:* Ginagamit ng modelong ito ang Tekken tokenizer imbes na ang mas karaniwang tiktoken. Dahil dito, mas maganda ang performance nito sa mas maraming wika at code.

- *Finetuning:* Ang base model ay available para sa finetuning. Nagbibigay ito ng mas malaking flexibility para sa mga sitwasyon na nangangailangan ng finetuning.

- *Native Function Calling* - Katulad ng Mistral Large, sinanay din ang modelong ito sa function calling. Dahil dito, natatangi ito bilang isa sa mga unang open source models na may ganitong kakayahan.


### Paghahambing ng mga Tokenizer

Sa halimbawang ito, titingnan natin kung paano hinahandle ng Mistral NeMo ang tokenization kumpara sa Mistral Large.

Parehong gumagamit ng iisang prompt ang dalawang halimbawa pero mapapansin mong mas kaunti ang tokens na ibinabalik ng NeMo kumpara sa Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Hindi dito nagtatapos ang pagkatuto, ipagpatuloy ang Paglalakbay

Pagkatapos mong matapos ang araling ito, bisitahin ang aming [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) para lalo pang mapalawak ang iyong kaalaman sa Generative AI!



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama’t nagsusumikap kami para sa kawastuhan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi eksaktong pagsasalin. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring lumitaw mula sa paggamit ng pagsasaling ito.
